# Load Libraries

In [56]:
import pandas as pd
import numpy as np
import random as rnd
import datetime

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
#sns.set_style('whitegrid')
%matplotlib inline

# plotly
import plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import cufflinks as cf
cf.go_offline()

from fastai.tabular.all import *

# SKlearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import *
from time import time


from tqdm import tqdm


# Load Data

In [57]:
filepath = "/Users/Kapil/PycharmProjects/stock-price"
train = pd.read_csv(filepath + "/data/train.csv")
test = pd.read_csv(filepath + "/data/test.csv")

In [58]:
SEED = 13
np.random.seed(SEED)

In [59]:
Y_train = train['Up']
X_train = train.drop(['Up'], axis = 1)

X_test = test

### Preprocessing Function

In [60]:
def preproc(df):
    #df['Date'] = pd.to_datetime(df.Date,format='%Y-%m-%d')
    df.index = df['Date']
    add_datepart(df, 'Date')
    df.drop('Elapsed', axis=1, inplace=True)
    return  df


X_train['Date'] = pd.to_datetime(X_train.Date,format='%Y-%m-%d')
X_train.index = X_train['Date']

plt.figure(figsize=(16,8))
plt.plot(X_train['Close'])
plt.title("Close price hisotry")
plt.savefig(filepath + "/figs/Close.jpg")

In [61]:
X_train = preproc(X_train)
X_train.head()


,Open,High,Low,Close,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start
Date,,,,,,,,,,,,,,,,
2004-01-02,50.85,50.91,50.62,50.72,2004,1,1,2,4,2,False,False,False,False,False,False
2004-01-05,50.75,50.84,50.61,50.79,2004,1,2,5,0,5,False,False,False,False,False,False
2004-01-06,50.87,51.26,50.86,51.15,2004,1,2,6,1,6,False,False,False,False,False,False
2004-01-07,51.45,51.54,51.29,51.51,2004,1,2,7,2,7,False,False,False,False,False,False
2004-01-08,51.15,51.33,51.02,51.28,2004,1,2,8,3,8,False,False,False,False,False,False


# Models


In [62]:
model = LogisticRegression()


In [63]:
kfolds = StratifiedKFold(n_splits=5, shuffle=True)
n_jobs = -1

### Parallel version

In [64]:
from joblib import Parallel, delayed
from sklearn.base import clone

def fit_and_score(model, train_index, valid_index, X, Y):
    """
    function to use in a joblib parallel env, takes the inputs and returns metrics
    return: (roc_train, roc_val, f1_train, f1_val)
    """
    # get X and y vectors
    x_train, x_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = Y.iloc[train_index], Y.iloc[valid_index]

    # over sample x
    #x_train, y_train = ros.fit_resample(x_train, y_train)

    # preprocess
#     x_train = ct.fit_transform(x_train)
#     x_valid = ct.transform(x_valid)

    # train model
    model.fit(x_train, y_train)

    # append scores
    y_train_preds = model.predict(x_train)
    y_valid_preds = model.predict(x_valid)

    f1_train = f1_score(y_train, y_train_preds)
    roc_train = roc_auc_score(y_train, model.predict_proba(x_train)[:,1])
    mcc_train = matthews_corrcoef(y_train, y_train_preds)

    f1_val = f1_score(y_valid, y_valid_preds)
    roc_val = roc_auc_score(y_valid, model.predict_proba(x_valid)[:,1])
    mcc_val = matthews_corrcoef(y_valid, y_valid_preds)

    log_line = 'Train score: ' + str(roc_train) + ' Validation score: ' + str(roc_val)
    tqdm.write(log_line)

    return roc_train, roc_val, f1_train, f1_val, mcc_train, mcc_val


In [65]:
st = time()

if n_jobs == 1:
    print('Running single process loop')
    res = []

    for train_index, valid_index in tqdm(kfolds.split(X_train, Y_train)):
        res.append(fit_and_score(clone(model), train_index, valid_index, X_train, Y_train))
else:
    print('Running parallel loop')
    parallel = Parallel(n_jobs=n_jobs, verbose=1)
    res = parallel(delayed(fit_and_score)(clone(model), train_idx, valid_idx, X_train, Y_train)
              for train_idx, valid_idx in kfolds.split(X_train, Y_train))


end = time()

print('Total time taken ', (end - st)/60, ' mins')

Running parallel loop


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Total time taken  0.07829741636912028  mins


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    4.7s finished


In [66]:
res = pd.DataFrame(res, columns=['ROC_train', 'ROC_val', 'F1_train', 'F1_val', 'MCC_train', 'MCC_val'])
res

,ROC_train,ROC_val,F1_train,F1_val,MCC_train,MCC_val
0,0.626634,0.599519,0.624928,0.621005,0.199156,0.180041
1,0.622820,0.594888,0.616550,0.595937,0.186235,0.112483
2,0.632853,0.594083,0.622874,0.568675,0.205028,0.115571
3,0.619514,0.610487,0.627070,0.585132,0.191965,0.144935
4,0.616256,0.644908,0.632883,0.643478,0.193107,0.188001


In [67]:
model.fit(X_train, Y_train)


/Users/Kapil/opt/anaconda3/envs/stock-price/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



LogisticRegression()

In [79]:
# model_pipe.roc_auc_score(X_train,Y_train)
X_test = preproc(X_test)
Y_train_preds = model.predict_proba(X_train)[:,1]
train_score = roc_auc_score(Y_train,Y_train_preds)

Y_test_preds = model.predict_proba(X_test)[:,1]
#test_score = roc_auc_score(Y_test,Y_test_preds)

print('Training score: ', train_score)
#print('Testing score: ', test_score)

KeyError: 'Date'

In [70]:
Y_test_preds.shape

(1200,)

In [71]:
Y_test.iloc[::6, :]



AttributeError: 'numpy.ndarray' object has no attribute 'iloc'

In [72]:
Y_test = pd.DataFrame(Y_test_preds)

In [76]:
Y_test.shape

(200, 1)

In [78]:
Y_test = Y_test.iloc[::6, :]
Y_test.head()


,0
0,0.395262
36,0.438776
72,0.355031
108,0.556924
144,0.570500


In [77]:
Y_test.to_csv('/Users/Kapil/Desktop/Y_test.csv')

